# Setup Environment and Read Data

In [ ]:
import torch
import numpy as np
import pandas as pd
import pickle
import copy
from tqdm import trange,tqdm
import torch.nn as nn
from torch.optim import Adam
from torch.optim.lr_scheduler import ReduceLROnPlateau
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import f1_score, classification_report, confusion_matrix

## Setup the Dataset

In [ ]:
DRIVE = "drive/MyDrive/Colab Notebooks/ELTE/DSLAB/swat/"
DRIVE = "swat/"
TRAIN_FILE_NAME = "SWaT_Dataset_Normal_v0 1.csv"
TRAIN_DATASET = DRIVE + TRAIN_FILE_NAME
TEST_FILE_NAME = "SWaT_Dataset_Attack_v0 1.csv"
TEST_DATASET = DRIVE + TEST_FILE_NAME

data_train = pd.read_csv(TRAIN_DATASET, header=None)
data_test = pd.read_csv(TEST_DATASET, header=None)

C:\Users\beren\AppData\Local\Temp\ipykernel_16372\3004897857.py:8: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51) have mixed types. Specify dtype option on import or set low_memory=False.
  data_train = pd.read_csv(TRAIN_DATASET, header=None)
C:\Users\beren\AppData\Local\Temp\ipykernel_16372\3004897857.py:9: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51) have mixed types. Specify dtype option on import or set low_memory=False.
  data_test = pd.read_csv(TEST_DATASET, header=None)


In [ ]:
data_train.replace("Normal", 0, inplace=True)
data_test.replace("Normal", 0, inplace=True)
data_test.replace("Attack", 1, inplace=True)
data_test.replace("A ttack", 1, inplace=True)

train = data_train.iloc[:, 1:].copy()
train = train.iloc[:, :-1].copy()
test = data_test.iloc[:, 1:].copy()
test = test.iloc[:, :-1].copy()

In [ ]:
true_anomalies = data_test[52].to_numpy()
true_anomalies = np.delete(true_anomalies, 0)
true_anomalies = true_anomalies.astype(int)

In [ ]:
true_anomalies

array([0, 0, 0, ..., 0, 0, 0])

In [ ]:
train

,1,2,3,4,5,6,7,8,9,10,...,42,43,44,45,46,47,48,49,50,51
0,FIT101,LIT101,MV101,P101,P102,AIT201,AIT202,AIT203,FIT201,MV201,...,FIT504,P501,P502,PIT501,PIT502,PIT503,FIT601,P601,P602,P603
1,2.470294,261.5804,2,2,1,244.3284,8.19008,306.101,2.471278,2,...,0,1,1,10.02948,0,4.277749,0.000256304,1,1,1
2,2.457163,261.1879,2,2,1,244.3284,8.19008,306.101,2.468587,2,...,0,1,1,10.02948,0,4.277749,0.000256304,1,1,1
3,2.439548,260.9131,2,2,1,244.3284,8.19008,306.101,2.467305,2,...,0,1,1,10.02948,0,4.277749,0.000256304,1,1,1
4,2.428338,260.285,2,2,1,244.3284,8.19008,306.101,2.466536,2,...,0,1,1,10.02948,0,4.277749,0.000256304,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
496796,2.460366,523.043,2,2,1,262.0161,8.396437,328.5055,2.442316,2,...,0.308619,2,1,250.8171,1.778105,189.8552,0.000128,1,1,1
496797,2.448836,522.9645,2,2,1,262.0161,8.396437,328.5055,2.442316,2,...,0.308619,2,1,250.8171,1.778105,189.5027,0.000128,1,1,1
496798,2.434744,522.886,2,2,1,262.0161,8.396437,328.6337,2.444879,2,...,0.308619,2,1,250.8171,1.778105,189.5027,0.000128,1,1,1
496799,2.428338,522.9252,2,2,1,262.0161,8.396437,328.6337,2.445391,2,...,0.308619,2,1,250.8171,1.649953,189.5027,0.000128,1,1,1


In [ ]:
test

,1,2,3,4,5,6,7,8,9,10,...,42,43,44,45,46,47,48,49,50,51
0,FIT101,LIT101,MV101,P101,P102,AIT201,AIT202,AIT203,FIT201,MV201,...,FIT504,P501,P502,PIT501,PIT502,PIT503,FIT601,P601,P602,P603
1,2.427057,522.8467,2,2,1,262.0161,8.396437,328.6337,2.445391,2,...,0.3077859,2,1,250.8652,1.649953,189.5988,0.000128152,1,1,1
2,2.446274,522.886,2,2,1,262.0161,8.396437,328.6337,2.445391,2,...,0.3077859,2,1,250.8652,1.649953,189.6789,0.000128152,1,1,1
3,2.489191,522.8467,2,2,1,262.0161,8.394514,328.6337,2.442316,2,...,0.3086186,2,1,250.8812,1.649953,189.6789,0.000128152,1,1,1
4,2.53435,522.9645,2,2,1,262.0161,8.394514,328.6337,2.442316,2,...,0.3086186,2,1,250.8812,1.649953,189.6148,0.000128152,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
449915,2.559972,519.5495,2,2,1,168.0979,8.638683,301.9226,2.459488,2,...,0.306569,2,1,251.1535,0.865024,189.022,0.0,1,1,1
449916,2.549082,520.4131,2,2,1,168.0979,8.638683,301.9226,2.459488,2,...,0.306569,2,1,251.0734,0.865024,188.9259,0.0,1,1,1
449917,2.531467,520.6878,2,2,1,168.0979,8.638683,301.9226,2.460129,2,...,0.306569,2,1,251.0734,0.865024,188.9259,0.0,1,1,1
449918,2.521218,520.7271,2,2,1,168.0979,8.638683,301.9226,2.460129,2,...,0.307978,2,1,251.0734,0.865024,188.9259,0.0,1,1,1


## Preprocess the Dataset

In [ ]:
# remove headers
if train.iloc[0].dtype == 'object':
    train = train[1:].copy()
    test = test[1:].copy()

# convert all columns to numeric
train = train.apply(pd.to_numeric, errors='coerce')
test = test.apply(pd.to_numeric, errors='coerce')

# interpolate/fill NaNs after conversion if necessary
train.interpolate(inplace=True)
train.bfill(inplace=True)
test.interpolate(inplace=True)
test.bfill(inplace=True)

# calculate variance, crop columns with variance close to zero.
variance_threshold = 1e-6
variances = train.var()
low_variance_cols = variances[variances < variance_threshold].index

print(f"Columns with very low variance: {list(low_variance_cols)}")

# drop low variance columns
train_filtered = train.drop(columns=low_variance_cols)
test_filtered = test.drop(columns=low_variance_cols)

print(f"Original number of features: {train.shape[1]}")
print(f"Number of features after removing low variance columns: {train_filtered.shape[1]}")

# egenerate the tensors and dataloaders with the filtered data
train_tensor = train_filtered.values
test_tensor = test_filtered.values

sequence_length = 30
sequences = []
for i in range(train_tensor.shape[0] - sequence_length + 1):
  sequences.append(train_tensor[i:i + sequence_length])

train_data, val_data = train_test_split(sequences, test_size=0.3, random_state=42, shuffle=False) # 70% train, 30% temp

test_sequences = []
for i in range(test_tensor.shape[0] - sequence_length + 1):
  test_sequences.append(test_tensor[i:i + sequence_length])

batch_size = 32
train_loader = DataLoader(dataset=train_data, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(dataset=val_data, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(dataset=test_sequences, batch_size=batch_size, shuffle=False)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Columns with very low variance: [5, 11, 12, 14, 16, 30, 32, 33, 44, 49, 51]
Original number of features: 51
Number of features after removing low variance columns: 40


In [ ]:
sequences[0].shape

(30, 40)

# Define the Networks

In [ ]:
input_dim = train_filtered.shape[1]
hidden_dim = 128
latent_dim = 32

In [ ]:
def save_model(model_name, model, input_dim, latent_dim, hidden_dim):
    model_state = {
        'input_dim':input_dim,
        'latent_dim':latent_dim,
        'hidden_dim':hidden_dim,
        'state_dict':model.state_dict()
    }
    torch.save(model_state, f'drive/MyDrive/Colab Notebooks/ELTE/DSLAB/{model_name}_swat.pth')

## AutoEncoder

In [ ]:
class FeedforwardEncoder(nn.Module):
    def __init__(self, input_dim, sequence_length, hidden_dim, latent_dim):
        super(FeedforwardEncoder, self).__init__()
        self.flatten_dim = input_dim * sequence_length
        self.encoder = nn.Sequential(
            nn.Linear(self.flatten_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, latent_dim)
        )

    def forward(self, x):
        x = x.view(x.size(0), -1)  # Flatten: (batch_size, sequence_length * input_dim)
        z = self.encoder(x)
        return z

class FeedforwardDecoder(nn.Module):
    def __init__(self, latent_dim, hidden_dim, output_dim, sequence_length):
        super(FeedforwardDecoder, self).__init__()
        self.output_dim = output_dim
        self.sequence_length = sequence_length
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, output_dim * sequence_length)
        )

    def forward(self, z):
        x_recon = self.decoder(z)
        return x_recon.view(z.size(0), self.sequence_length, self.output_dim)

class AE(nn.Module):
    def __init__(self, input_dim, sequence_length, hidden_dim, latent_dim, device='cpu'):
        super(AE, self).__init__()
        self.encoder = FeedforwardEncoder(input_dim, sequence_length, hidden_dim, latent_dim).to(device)
        self.decoder = FeedforwardDecoder(latent_dim, hidden_dim, input_dim, sequence_length).to(device)

    def forward(self, x):
        z = self.encoder(x)
        x_recon = self.decoder(z)
        return x_recon

In [ ]:
def loss_function_ae(x, x_hat):
    return nn.functional.mse_loss(x_hat, x, reduction='sum')

In [ ]:
model_ae = AE(input_dim=input_dim,
                hidden_dim=hidden_dim,
                latent_dim=latent_dim,
                sequence_length=sequence_length,
                device=device).to(device)
optimizer_ae = Adam(model_ae.parameters(), lr=1e-3)
scheduler_ae = ReduceLROnPlateau(optimizer_ae, 'min', patience=5, factor=0.1)

/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


## LSTM AutoEncoder

In [ ]:
class LSTMEncoder(nn.Module):
    def __init__(self, input_dim, hidden_dim, latent_dim, num_layers=1):
        super(LSTMEncoder, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers=num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, latent_dim)

    def forward(self, x):
        _, (h_n, _) = self.lstm(x)
        h = h_n[-1]  # Take last layer's hidden state
        z = self.fc(h)
        return z

class LSTMDecoder(nn.Module):
    def __init__(self, latent_dim, hidden_dim, output_dim, sequence_length, num_layers=1):
        super(LSTMDecoder, self).__init__()
        self.sequence_length = sequence_length
        self.latent_to_hidden = nn.Linear(latent_dim, hidden_dim)
        self.lstm = nn.LSTM(hidden_dim, hidden_dim, num_layers=num_layers, batch_first=True)
        self.output_layer = nn.Linear(hidden_dim, output_dim)

    def forward(self, z):
        hidden = self.latent_to_hidden(z).unsqueeze(1).repeat(1, self.sequence_length, 1)
        out, _ = self.lstm(hidden)
        return self.output_layer(out)


class LSTMAE(nn.Module):
    def __init__(self, input_dim, hidden_dim, latent_dim, sequence_length, num_layers=1, device='cpu'):
        super(LSTMAE, self).__init__()
        self.encoder = LSTMEncoder(input_dim, hidden_dim, latent_dim, num_layers).to(device)
        self.decoder = LSTMDecoder(latent_dim, hidden_dim, input_dim, sequence_length, num_layers).to(device)

    def forward(self, x):
        z = self.encoder(x)
        x_recon = self.decoder(z)
        return x_recon

In [ ]:
def loss_function_lstm_ae(x, x_hat):
    return nn.functional.mse_loss(x_hat, x, reduction='sum')

In [ ]:
model_lstm_ae = LSTMAE(input_dim=input_dim,
                hidden_dim=hidden_dim,
                latent_dim=latent_dim,
                sequence_length=sequence_length,
                num_layers=1,
                device=device).to(device)
optimizer_lstm_ae = Adam(model_lstm_ae.parameters(), lr=1e-3)
scheduler_lstm_ae = ReduceLROnPlateau(optimizer_lstm_ae, 'min', patience=5, factor=0.1)

## LSTM Variational AutoEncoder

In [ ]:
class LSTMEncoder(nn.Module):
    def __init__(self, input_dim, hidden_dim, latent_dim, num_layers=1):
        super(LSTMEncoder, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers=num_layers, batch_first=True)
        self.fc_mean = nn.Linear(hidden_dim, latent_dim)
        self.fc_logvar = nn.Linear(hidden_dim, latent_dim)

    def forward(self, x):
        _, (h_n, _) = self.lstm(x)  # h_n: (num_layers, batch, hidden_dim)
        h = h_n[-1]  # take the output of the last layer
        return self.fc_mean(h), self.fc_logvar(h)


class LSTMDecoder(nn.Module):
    def __init__(self, latent_dim, hidden_dim, output_dim, sequence_length, num_layers=1):
        super(LSTMDecoder, self).__init__()
        self.sequence_length = sequence_length
        self.latent_to_hidden = nn.Linear(latent_dim, hidden_dim)
        self.lstm = nn.LSTM(hidden_dim, hidden_dim, num_layers=num_layers, batch_first=True)
        self.output_layer = nn.Linear(hidden_dim, output_dim)

    def forward(self, z):
        # Repeat z for each timestep
        hidden = self.latent_to_hidden(z).unsqueeze(1).repeat(1, self.sequence_length, 1)
        out, _ = self.lstm(hidden)
        return self.output_layer(out)


class LSTMVAE(nn.Module):
    def __init__(self, input_dim, hidden_dim, latent_dim, sequence_length, num_layers=1, device='cpu'):
        super(LSTMVAE, self).__init__()
        self.encoder = LSTMEncoder(input_dim, hidden_dim, latent_dim, num_layers).to(device)
        self.decoder = LSTMDecoder(latent_dim, hidden_dim, input_dim, sequence_length, num_layers).to(device)

    def reparameterize(self, mean, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mean + eps * std

    def forward(self, x):
        mean, logvar = self.encoder(x)
        z = self.reparameterize(mean, logvar)
        x_recon = self.decoder(z)
        return x_recon, mean, logvar

In [ ]:
def loss_function_lstm_vae(x, x_hat, mean, log_var):
    reproduction_loss = nn.functional.mse_loss(x_hat, x, reduction='sum')
    KLD = - 0.5 * torch.sum(1+ log_var - mean.pow(2) - log_var.exp())

    return reproduction_loss + KLD

In [ ]:
model_lstm_vae = LSTMVAE(input_dim=input_dim,
                hidden_dim=hidden_dim,
                latent_dim=latent_dim,
                sequence_length=sequence_length,
                num_layers=1,
                device=device).to(device)
optimizer_lstm_vae = Adam(model_lstm_vae.parameters(), lr=1e-3)
scheduler_lstm_vae = ReduceLROnPlateau(optimizer_lstm_vae, 'min', patience=5, factor=0.1)

# Train

In [ ]:
def train_model(model, train_loader, val_loader, optimizer, loss_fn, scheduler, variational=False, num_epochs=10, device='cpu'):
    torch.cuda.empty_cache()
    train_losses = []
    val_losses = []

    early_stop_tolerant_count = 0
    early_stop_tolerant = 10
    best_loss = float('inf')
    best_model_wts = copy.deepcopy(model.state_dict())

    for epoch in range(num_epochs):
        train_loss = 0.0
        model.train()
        for batch in train_loader:
            batch = torch.tensor(batch, dtype=torch.float32).to(device)

            optimizer.zero_grad()

            if variational:
                recon_batch, mean, logvar = model(batch)
                loss = loss_fn(recon_batch, batch, mean, logvar)
            else:
                recon_batch = model(batch)
                loss = loss_fn(batch, recon_batch)

            loss.backward()
            optimizer.step()
            train_loss += loss.item()

        train_loss /= len(train_loader)
        train_losses.append(train_loss)

        # Validation
        model.eval()
        valid_loss = 0.0
        with torch.no_grad():
            for batch in val_loader:
                batch = torch.tensor(batch, dtype=torch.float32).to(device)
                if variational:
                    recon_batch, mean, logvar = model(batch)
                    loss = loss_fn(recon_batch, batch, mean, logvar)
                else:
                    recon_batch = model(batch)
                    loss = loss_fn(batch, recon_batch)
                valid_loss += loss.item()

        valid_loss /= len(val_loader)
        val_losses.append(valid_loss)

        scheduler.step(valid_loss)

        if valid_loss < best_loss:
            best_loss = valid_loss
            best_model_wts = copy.deepcopy(model.state_dict())
            early_stop_tolerant_count = 0
        else:
            early_stop_tolerant_count += 1

        print(f"Epoch {epoch+1:04d}: train loss {train_loss:.4f}, valid loss {valid_loss:.4f}")

        if early_stop_tolerant_count >= early_stop_tolerant:
            print("Early stopping triggered.")
            break

    model.load_state_dict(best_model_wts)
    print("Finished Training.")
    return train_losses, val_losses

## AutoEncoder

In [ ]:
train_losses_ae, val_losses_ae = train_model(model_ae, train_loader, val_loader, optimizer_ae, loss_function_ae, scheduler_ae, False, num_epochs=100, device=device)

<ipython-input-30-eb483228514d>:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  batch = torch.tensor(batch, dtype=torch.float32).to(device)
<ipython-input-30-eb483228514d>:38: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  batch = torch.tensor(batch, dtype=torch.float32).to(device)


Epoch 0001: train loss 3746593.9785, valid loss 85726.3890
Epoch 0002: train loss 144053.5055, valid loss 50729.2204
Epoch 0003: train loss 112825.0454, valid loss 58146.6984
Epoch 0004: train loss 96323.6549, valid loss 46626.7136
Epoch 0005: train loss 91106.0577, valid loss 107728.0100
Epoch 0006: train loss 76006.1068, valid loss 45092.2299
Epoch 0007: train loss 74094.3563, valid loss 42665.6435
Epoch 0008: train loss 66226.1873, valid loss 26412.4232
Epoch 0009: train loss 63277.1252, valid loss 29598.9246
Epoch 0010: train loss 56904.8939, valid loss 22018.2232
Epoch 0011: train loss 58788.0369, valid loss 20833.0376
Epoch 0012: train loss 53234.3205, valid loss 19124.8727
Epoch 0013: train loss 55050.9334, valid loss 29846.3934
Epoch 0014: train loss 59619.5085, valid loss 22179.9815
Epoch 0015: train loss 52749.3262, valid loss 23550.7394
Epoch 0016: train loss 52603.2402, valid loss 46582.5573
Epoch 0017: train loss 49593.7898, valid loss 20458.6073
Epoch 0018: train loss 503

In [ ]:
save_model('ae', model_ae, input_dim, latent_dim, hidden_dim)

## LSTM AutoEncoder

In [ ]:
train_losses_lstm_ae, val_losses_lstm_ae = train_model(model_lstm_ae, train_loader, val_loader, optimizer_lstm_ae, loss_function_lstm_ae, scheduler_lstm_ae, False, num_epochs=100, device=device)

C:\Users\beren\AppData\Local\Temp\ipykernel_16372\4122512765.py:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  batch = torch.tensor(batch, dtype=torch.float32).to(device)
C:\Users\beren\AppData\Local\Temp\ipykernel_16372\4122512765.py:38: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  batch = torch.tensor(batch, dtype=torch.float32).to(device)


Epoch 0001: train loss 484686602.6281, valid loss 26659638.2477
Epoch 0002: train loss 24587868.0093, valid loss 14049063.8727
Epoch 0003: train loss 17806276.3389, valid loss 9828138.6842
Epoch 0004: train loss 7483515.6929, valid loss 5516648.1820
Epoch 0005: train loss 5230576.3371, valid loss 3885575.0685
Epoch 0006: train loss 4107833.6561, valid loss 5309432.7394
Epoch 0007: train loss 11212954.2899, valid loss 6486255.1841
Epoch 0008: train loss 13973055.9771, valid loss 8971457.0284
Epoch 0009: train loss 12833585.8724, valid loss 9557031.2028
Epoch 0010: train loss 13604243.8522, valid loss 9152343.7176
Epoch 0011: train loss 14004243.5311, valid loss 9978875.8604
Epoch 0012: train loss 10458566.7293, valid loss 8960171.4019
Epoch 0013: train loss 10301433.4261, valid loss 8925261.8582
Epoch 0014: train loss 10292712.6537, valid loss 8950015.0371
Epoch 0015: train loss 10299603.8725, valid loss 8966927.4015
Early stopping triggered.
Finished Training.


## LSTM Variational AutoEncoder

In [ ]:
train_losses_lstm_vae, val_losses_lstm_vae = train_model(model_lstm_vae, train_loader, val_loader, optimizer_lstm_vae, loss_function_lstm_vae, scheduler_lstm_vae, True, num_epochs=100, device=device)

C:\Users\beren\AppData\Local\Temp\ipykernel_16372\4122512765.py:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  batch = torch.tensor(batch, dtype=torch.float32).to(device)
C:\Users\beren\AppData\Local\Temp\ipykernel_16372\4122512765.py:38: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  batch = torch.tensor(batch, dtype=torch.float32).to(device)


Epoch 0001: train loss 479709263.9279, valid loss 26761825.6954
Epoch 0002: train loss 31093425.7893, valid loss 19361644.7293
Epoch 0003: train loss 28257835.5092, valid loss 19789416.6956
Epoch 0004: train loss 25822521.8918, valid loss 20726859.3412
Epoch 0005: train loss 24667014.4285, valid loss 20577813.0159
Epoch 0006: train loss 25171671.9553, valid loss 20470875.3184
Epoch 0007: train loss 23701195.9420, valid loss 20459874.7399
Epoch 0008: train loss 25742572.5053, valid loss 20556115.4575
Epoch 0009: train loss 23834008.9334, valid loss 20488681.8278
Epoch 0010: train loss 23240646.4510, valid loss 20492438.1242
Epoch 0011: train loss 22926572.5109, valid loss 20477925.8783
Epoch 0012: train loss 22553609.6046, valid loss 20477209.7088
Early stopping triggered.
Finished Training.


# Evaluation

In [ ]:
def evaluate_model(model, variational, test_loader, device, loss_fn, percentile_threshold=90):
    model.eval()
    anomaly_scores = []

    with torch.no_grad():
        for batch in test_loader:
            batch = torch.tensor(batch, dtype=torch.float32).to(device)

            batch_scores = []
            for i in range(batch.shape[0]):  # Iterate through each sequence in the batch
                sequence = batch[i].unsqueeze(0)  # Shape: (1, seq_len, features)
                if variational:
                    recon_sequence, mean, logvar = model(sequence)
                    loss = loss_fn(recon_sequence, sequence, mean, logvar)
                else:
                    recon_sequence = model(sequence)
                    loss = loss_fn(sequence, recon_sequence)
                batch_scores.append(loss.item())

            anomaly_scores.extend(batch_scores)

    # Calculate threshold and identify anomalies
    threshold = np.percentile(anomaly_scores, percentile_threshold)
    anomaly_indices = [i for i, score in enumerate(anomaly_scores) if score > threshold]
    return anomaly_indices

In [ ]:
def calculate_f1_score(anomaly_indices, true_anomalies):
    # Create a binary array representing predicted anomalies
    predicted_anomalies = np.zeros_like(true_anomalies)
    for index in anomaly_indices:
        if index < len(predicted_anomalies):  # Check index bounds
          predicted_anomalies[index] = 1

    # Calculate the F1 score
    f1 = f1_score(true_anomalies, predicted_anomalies)
    return f1, predicted_anomalies

## AutoEncoder

In [ ]:
anomalies_ae = evaluate_model(model_ae, False, test_loader, device, loss_function_ae, 90)

<ipython-input-33-6ed720cb0935>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  batch = torch.tensor(batch, dtype=torch.float32).to(device)


In [ ]:
f1_ae, predicted_anomalies_ae = calculate_f1_score(anomalies_ae, true_anomalies)
print(f"F1 Score: {f1_ae}")

F1 Score: 0.7452063045878928


In [ ]:
print(classification_report(true_anomalies, predicted_anomalies_ae))

              precision    recall  f1-score   support

           0       0.96      0.98      0.97    395298
           1       0.82      0.68      0.75     54621

    accuracy                           0.94    449919
   macro avg       0.89      0.83      0.86    449919
weighted avg       0.94      0.94      0.94    449919



In [ ]:
print(confusion_matrix(true_anomalies, predicted_anomalies_ae))

[[387424   7874]
 [ 17506  37115]]


## LSTM AutoEncoder

In [ ]:
anomalies_lstm_ae = evaluate_model(model_lstm_ae, False, test_loader, device, loss_function_lstm_ae, 90)

C:\Users\beren\AppData\Local\Temp\ipykernel_16372\2047070644.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  batch = torch.tensor(batch, dtype=torch.float32).to(device)


In [ ]:
f1_lstm_ae, predicted_anomalies_lstm_ae = calculate_f1_score(anomalies_lstm_ae, true_anomalies)
print(f"F1 Score: {f1_lstm_ae}")

F1 Score: 0.7037044473446441


In [ ]:
print(classification_report(true_anomalies, predicted_anomalies_lstm_ae))

              precision    recall  f1-score   support

           0       0.95      0.97      0.96    395298
           1       0.78      0.64      0.70     54621

    accuracy                           0.93    449919
   macro avg       0.87      0.81      0.83    449919
weighted avg       0.93      0.93      0.93    449919



In [ ]:
print(confusion_matrix(true_anomalies, predicted_anomalies_lstm_ae))

[[385357   9941]
 [ 19573  35048]]


## LSTM Variational AutoEncoder

In [ ]:
anomalies_lstm_vae = evaluate_model(model_lstm_vae, True, test_loader, device, loss_function_lstm_vae, 90)

C:\Users\beren\AppData\Local\Temp\ipykernel_16372\2047070644.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  batch = torch.tensor(batch, dtype=torch.float32).to(device)


In [ ]:
f1_lstm_vae, predicted_anomalies_lstm_vae = calculate_f1_score(anomalies_lstm_vae, true_anomalies)
print(f"F1 Score: {f1_lstm_vae}")

F1 Score: 0.7031623330990865


In [ ]:
print(classification_report(true_anomalies, predicted_anomalies_lstm_vae))

              precision    recall  f1-score   support

           0       0.95      0.97      0.96    395298
           1       0.78      0.64      0.70     54621

    accuracy                           0.93    449919
   macro avg       0.87      0.81      0.83    449919
weighted avg       0.93      0.93      0.93    449919



In [ ]:
print(confusion_matrix(true_anomalies, predicted_anomalies_lstm_vae))

[[385330   9968]
 [ 19600  35021]]
